In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import os
from rasterio import features
import rasterio
import matplotlib.pyplot as plt
import ahp

In [2]:
%load_ext autoreload
%autoreload 2

In [77]:
path_files = r'/home/jta/Documentos/raster_variables/SIG'
files = os.listdir(path_files)

In [78]:
path = r'Vento-Tabla General de Variables Multidimensionales.xlsx'
df_vars = pd.read_excel(path,)

In [79]:
def name_var(
        cod:str,
        df=df_vars
    ):
    mask = df['Código'] == cod
    var = df[mask].iloc[0,-1]
    return var


In [80]:
# Creacion diccionario info
dict_vars = dict()
# Creacion de geodataframes
for file in files:
    # Extraccion variable
    var = name_var(file)
    # Creacion de geodaframe
    full_path = os.path.join(
        path_files,
        file
    )
    gdf = gpd.read_file(full_path)
    # Almacenamiento
    dict_vars[var] = gdf

/home/jta/miniconda3/envs/cds/lib/python3.14/site-packages/pyogrio/raw.py:200: RuntimeWarning: organizePolygons() received an unexpected geometry.  Either a polygon with interior rings, or a polygon with less than 4 points, or a non-Polygon geometry.  Return arguments as a collection.
  return ogr_read(
/home/jta/miniconda3/envs/cds/lib/python3.14/site-packages/pyogrio/raw.py:200: RuntimeWarning: Geometry of polygon of fid 1655 cannot be translated to Simple Geometry. All polygons will be contained in a multipolygon.
  return ogr_read(


In [81]:
# Lectura de datos raster
path_raster = r'/home/jta/Documentos/raster_variables/COL_wind-speed_10m.tif'
da_wind = rasterio.open(path_raster)

## Rasterizacion de las capas binarias

In [96]:
xr_wind = xr.open_dataset(path_raster)
lon = xr_wind.x.values
lat = xr_wind.y.values
# Coordenadas de los datos
coords = {
    'y':lat,
    'x': lon,
}

In [97]:
dict_rasters = dict()
for var,gdf in dict_vars.items():
    # conversion de crs al raster
    crs_raster = da_wind.crs
    gdf_reproj = gdf.to_crs(crs_raster) 
    #Extraccion geometria
    geom = gdf_reproj.geometry
    # rasterizacion en binarios
    rasterized = features.rasterize(
        geom,
        out_shape=da_wind.shape,
        fill=0,
        out=None,
        transform=da_wind.transform,
        all_touched=False,
        default_value=1,
        dtype=None
    )
    # conversion como datarray
    da_var = xr.DataArray(
        data=rasterized,
        coords=coords
    )

    # almacenamiento
    dict_rasters[var] = da_var


/home/jta/miniconda3/envs/cds/lib/python3.14/site-packages/rasterio/features.py:364: ShapeSkipWarning: Invalid or empty shape None at index 2396 will not be rasterized.
  warnings.warn(
/home/jta/miniconda3/envs/cds/lib/python3.14/site-packages/rasterio/features.py:364: ShapeSkipWarning: Invalid or empty shape None at index 2397 will not be rasterized.
  warnings.warn(
/home/jta/miniconda3/envs/cds/lib/python3.14/site-packages/rasterio/features.py:364: ShapeSkipWarning: Invalid or empty shape None at index 2398 will not be rasterized.
  warnings.warn(
/home/jta/miniconda3/envs/cds/lib/python3.14/site-packages/rasterio/features.py:364: ShapeSkipWarning: Invalid or empty shape None at index 2399 will not be rasterized.
  warnings.warn(
/home/jta/miniconda3/envs/cds/lib/python3.14/site-packages/rasterio/features.py:364: ShapeSkipWarning: Invalid or empty shape None at index 2400 will not be rasterized.
  warnings.warn(
/home/jta/miniconda3/envs/cds/lib/python3.14/site-packages/rasterio/fe

In [98]:
# Almacenamiento de los datos
path_guardado = r'/home/jta/Documentos/raster_variables/raster_data'
for var,raster in dict_rasters.items():
    # Elimina los espacios en blanco
    name_var = var.replace(' ','_')
    # Crea el nombre .tif
    var_save = name_var + '.tif'
    # path guardado
    path_var = os.path.join(path_guardado,var_save)
    # exporta los datos
    raster.rio.to_raster(path_var)

## Aplicación AHP

### Identificar zonas viables
Se multiplica la capa de vientos por las variables excluyentes

In [7]:
path_excluyentes = r'/home/jta/Documentos/raster_variables/var_excluyentes'
files = os.listdir(path_excluyentes)
files_excluyente = [os.path.join(path_excluyentes,file) for file in files]

In [8]:
# Diccionario de almacenamiento
vars_conv = dict()
# Lectura de los datos
for idx,file in enumerate(files_excluyente):
    #nombre_variable
    var_name = files[idx].split('.')[0]
    da = xr.open_dataarray(file)
    # Conversion a array de numpy
    da_values = da[0,:,:].values
    # Conversion a faltantes
    da_conversion = np.where(
        da_values == 1, 
        np.nan,
        da_values,
    )
    vars_conv[var_name] = da_conversion

In [10]:
# Lectura de datos de velocidad

vel_da = xr.open_dataarray('COL_wind-speed_10m.tif')
vel_da = vel_da[0,:,:].values
# Conversion de zonas potenciales
vel_binario = np.where(vel_da >= 5.0,1,0)
## Multiplicacion de capas
zonas_viables = vel_binario + (vel_binario * vars_conv['Cienagas'] * vars_conv['Humedales_OFFSHORE'] * \
                vars_conv['Lagunas'] * vars_conv['Pantanos'] * vars_conv['RUNAP'] * \
                vars_conv['Reservas_y_zonas_de_amortiguamiento_de_páramos'] * \
                vars_conv['Ríos_(Drenaje_doble)'])

### Organizar capas ráster
Donde no haya presencia se asigna valor de 1, caso contrario 0

In [11]:
path_penalizantes = '/home/jta/Documentos/raster_variables/raster_data'
files_pen = os.listdir(path_penalizantes)
files_path = list()
# path de los archivos
for file in files_pen:
    if file.split('.')[-1] == 'tif':
        file_path = os.path.join(
            path_penalizantes,
            file
        )
        files_path.append(file_path)

In [12]:
dict_vars_pen = dict()
# Creacion de datarrays 
for file in files_path:
    # nombre de la variable
    var_name = file.split('/')[-1].split('.')[0]
    da = xr.open_dataarray(file)
    da_var = da[0,:,:].values
    # INversion de los datos
    da_invert = np.where(da_var == 1,0,1)
    # almacenamiento de los datos
    dict_vars_pen[var_name] = da_invert


### Aplicación método AHP

In [13]:
from ahp import Transformer

In [18]:
# Numero de variables
num_vars = len(dict_vars_pen.keys())

In [ ]:
ore = ahp.consistency_rate(matrix)

matrix 1 score:  25.140784485605984   0.37616397060106926


In [133]:
ahp = Transformer()
matrix = ahp.simulate_matrices(1,num_vars)
score = ahp.consistency_rate(matrix)[0]
while np.abs(score) > 0.22:
    matrix = ahp.simulate_matrices(1,num_vars)
    score = ahp.consistency_rate(matrix)[0]

matrix 1 score:  24.49380663828101   0.3495393678304942
matrix 1 score:  24.591356000184962   0.3535537448635787
matrix 1 score:  27.238080783161347   0.46247246021240107
matrix 1 score:  25.175431725670087   0.3775897829493863
matrix 1 score:  25.93761056167154   0.408955167146977
matrix 1 score:  24.39212451678118   0.3453549183860568
matrix 1 score:  25.387904408859676   0.38633351476788785
matrix 1 score:  26.285310266271644   0.42326379696591127
matrix 1 score:  25.10396563601162   0.37464879160541636
matrix 1 score:  26.105225471259736   0.41585290005184095
matrix 1 score:  24.06251093503696   0.3317905734583111
matrix 1 score:  23.453084045061722   0.30671127757455646
matrix 1 score:  25.126358249448163   0.3755702983312001
matrix 1 score:  25.218859013796205   0.37937691414799196
matrix 1 score:  23.29315250267683   0.3001297326204456
matrix 1 score:  24.742149684603092   0.35975924627996264
matrix 1 score:  24.39651499466651   0.34553559648833376
matrix 1 score:  23.3809884706

In [136]:
pesos = ahp.calculate_weights(matrix)

np.float64(0.06856731999228607)